# Intro to Cython

Adapted from <https://github.com/jni/aspp-2017-cython>  by Juan-Nunez Iglesias. Distributed under BSD 3-clause "New" or "Revised" License. Check out the materials there for more on parallelizing code with Cython, and wrapping C/C++ code.

## Outline:

* Speed up Python code
* Interact with NumPy arrays

# Part 1: speed up your Python code

We want to integrate the function $f(x) = x^4 - 3x$.

In [1]:
def f(x):
    y = x**4 - 3*x
    return y
    
def integrate_f(a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f(a) * dx2
    for i in range(1, n):
        s += f(a + i * dx) * dx
    s += f(b) * dx2
    return s

Now, let's time this:

In [2]:
%timeit integrate_f(-100, 100, int(1e5))

57.6 ms ± 6.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Not too bad, but this can add up. Let's see if Cython can do better:

In [3]:
%load_ext cython

In [4]:
%%cython

def f2(x):
    y = x**4 - 3*x
    return y
    
def integrate_f2(a, b, n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f2(a) * dx2
    for i in range(1, n):
        s += f2(a + i * dx) * dx
    s += f2(b) * dx2
    return s

In [5]:
%timeit integrate_f2(-100, 100, int(1e5))

40.2 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


That's a little bit faster, which is nice since all we did was to call Cython on the exact same code. But can we do better?

In [6]:
%%cython

def f3(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f3(double a, double b, int n):
    dx = (b - a) / n
    dx2 = dx / 2
    s = f3(a) * dx2
    for i in range(1, n):
        s += f3(a + i * dx) * dx
    s += f3(b) * dx2
    return s

In [7]:
%timeit integrate_f3(-100, 100, int(1e5))

26 ms ± 1.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The final bit of "easy" Cython optimization is "declaring" the variables inside the function:

In [8]:
%%cython

def f4(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f4(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s = f4(a) * dx2
        int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

In [9]:
%timeit integrate_f4(-100, 100, int(1e5))

15.1 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%cython

def f4(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f4(double a, double b, int n):
    cdef double dx = (b - a) / n
    cdef double dx2 = dx / 2
    cdef double s = f4(a) * dx2
    cdef int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

4X speedup with so little effort is pretty nice. What else can we do?

Cython has a nice "-a" flag (for annotation) that can provide clues about why your code is slow.

In [11]:
%%cython -a

def f4(double x):
    y = x**4 - 3*x
    return y
    
def integrate_f4(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s = f4(a) * dx2
        int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

That's a lot of yellow still! How do we reduce this?

### Exercise: change the `f4` declaration to C

In [12]:
%%cython -a

cdef f4(double x):
    cdef double y
    y = x**4 - 3*x
    return y
    
def integrate_f5(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double dx2 = dx / 2
        double s = f4(a) * dx2
        int i = 0
    for i in range(1, n):
        s += f4(a + i * dx) * dx
    s += f4(b) * dx2
    return s

In [13]:
%timeit integrate_f5(-100, 100, 10**5) 

10.9 ms ± 63.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Part 2: work with NumPy arrays

The above is a very small subset of Python. Most scientific application deal not with single values, but with arrays of data.

In [14]:
import numpy as np

def mean3filter(arr):
    arr_out = np.empty_like(arr)
    for i in range(1, arr.shape[0] - 1):
        arr_out[i] = np.sum(arr[i-1 : i+2]) / 3
    arr_out[0] = (arr[0] + arr[1]) / 2
    arr_out[-1] = (arr[-1] + arr[-2]) / 2
    return arr_out

In [15]:
r = np.random.rand(10**5)

In [16]:
%timeit mean3filter(r)

357 ms ± 3.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%cython
import cython
import numpy as np

@cython.boundscheck(False)
def mean3filter2(double[::1] arr):
    cdef double[::1] arr_out = np.empty_like(arr)
    cdef int i
    for i in range(1, arr.shape[0]-1):
        arr_out[i] = np.sum(arr[i-1 : i+2]) / 3
    arr_out[0] = (arr[0] + arr[1]) / 2
    arr_out[-1] = (arr[-1] + arr[-2]) / 2
    return np.asarray(arr_out)

In [18]:
%timeit mean3filter2(r)

819 ms ± 79.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Rubbish! How do we fix this?

### Exercise: use `%%cython -a` to speed up the code

In [19]:
%%cython -a
import cython
import numpy as np

@cython.boundscheck(False)
def mean3filter3(double[::1] arr, double[::1] arr_out):
    cdef int i
    for i in range(1, arr.shape[0]-1):
        arr_out[i] = (arr[i-1] + arr[i] + arr[i+1]) / 3
    arr_out[0] = (arr[0] + arr[1]) / 2
    arr_out[-1] = (arr[-1] + arr[-2]) / 2

In [20]:
arr_out = np.zeros_like(r)
%timeit mean3filter3(r, arr_out)

479 µs ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
